In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import mysql.connector

db = mysql.connector.connect(host = "host_name",
                            username = "user_name",
                            password = "password",
                            database = "database")

cur = db.cursor()

In [ ]:
import numpy as np

### Query 01 :

In [ ]:
query1 = """ select distinct customer_city as cities from customers """

cur.execute(query1)

data1 = cur.fetchall()


df1 = pd.DataFrame(data1, columns = ["Cities"])
df1.head()

### Query 02 :

In [ ]:
query2 = """ select count(order_id) , year(order_purchase_timestamp) as order_year
from orders
where order_purchase_timestamp = 2017; """

cur.execute(query2)

data2 = cur.fetchall()


df2 = pd.DataFrame(data2, columns = ["Total_orders","year"])
df2

### Query 03 :

In [ ]:
query3 = """ select upper(products.product_category), round(sum(payments.payment_value),2) as total_sales
from products join order_items
on products.product_id = order_items.product_id
join payments
on order_items.order_id = payments.order_id
group by products.product_category; """

cur.execute(query3)

data3 = cur.fetchall()

df3 = pd.DataFrame(data3, columns = ["Category","Total_sales"])
df3.head()

### Query 04 :

In [ ]:
query4 = """ select ((sum(case when payment_installments >= 1 then 1 else 0 end) * 100) / count(*)) as percentage_for_installments
from payments; """

cur.execute(query4)

data4 = cur.fetchall()

df4 = pd.DataFrame(data4, columns = ["percentage_for_installments"])
df4

### Query 05 :

In [ ]:
query5 = """ select customer_state, count(customer_id) as number_of_customers
from customers
group by customer_state """

cur.execute(query5)

data5 = cur.fetchall()


df5 = pd.DataFrame(data5, columns = ["customer_state", "number_of_customers"])
df5.head()

### Query 06 :

In [ ]:
query6 = """ SELECT 
    YEAR(order_purchase_timestamp) AS order_year,
    MONTHNAME(order_purchase_timestamp) AS month,
    COUNT(order_id) AS total_orders
FROM
    orders
WHERE
    order_purchase_timestamp = 2018
GROUP BY MONTHNAME(order_purchase_timestamp)
ORDER BY MONTH(order_purchase_timestamp); """

cur.execute(query6)

data6 = cur.fetchall()

df6 = pd.DataFrame(data6, columns = ["order_year", "Month", "Total_orders"])
df6

### Query 07(1) :

In [ ]:
query7 = """ SELECT 
    city,
    ROUND(AVG(order_count), 2) AS average_number_of_products_per_order
FROM
    (SELECT 
        orders.order_id,
            customers.customer_city AS city,
            COUNT(order_items.product_id) AS order_count
    FROM
        order_items
    JOIN orders ON order_items.order_id = orders.order_id
    JOIN customers ON orders.customer_id = customers.customer_id
    GROUP BY orders.order_id , customers.customer_city) subquery
GROUP BY city; """

cur.execute(query7)

data7 = cur.fetchall()

df7 = pd.DataFrame(data7, columns = ["City", "Average_number_of_products_per_order"])
df7.head()

### Query 07(2) :

In [ ]:
query7_2 = """ with count_per_order as
(select orders.order_id, 
orders.customer_id, 
count(order_items.order_id) AS order_count
from orders join order_items
on orders.order_id = order_items.order_id
group by orders.order_id, orders.customer_id)

select customers.customer_city, round(avg(count_per_order.order_count),2) as average_number_of_products_per_order
from customers join count_per_order
on customers.customer_id = count_per_order.customer_id
group by customers.customer_city; """

cur.execute(query7_2)

data7_2 = cur.fetchall()

df7_2 = pd.DataFrame(data7_2, columns = ["City", "Average_number_of_products_per_order"])
df7_2.head()

### Query 08 :

In [ ]:
query8 = """ SELECT 
    UPPER(products.product_category) as category,
    round((SUM(payments.payment_value)  / (select sum(payments.payment_value) from payments)) * 100, 2) AS category_wise_total_sales
FROM
    products
        JOIN
    order_items ON products.product_id = order_items.product_id
        JOIN
    payments ON order_items.order_id = payments.order_id
GROUP BY category; """

cur.execute(query8)

data8 = cur.fetchall()

df8 = pd.DataFrame(data8, columns = ["category", "category_wise_total_sales_in_percentage"])
df8.head()

### Query 09 :

In [ ]:
query9 = """ SELECT 
    products.product_category,
    COUNT(order_items.order_id) AS total_orders,
    ROUND(AVG(order_items.price), 2) AS average_price
FROM
    products
        JOIN
    order_items ON products.product_id = order_items.product_id
GROUP BY products.product_category """

cur.execute(query9)

data9 = cur.fetchall()

df9 = pd.DataFrame(data9, columns = ["Category", "Total Orders", "Average Price"])
df9.head()

In [ ]:
arr1 = df9["Total Orders"]
arr2 = df9["Average Price"]

a= np.corrcoef([arr1,arr2])
print("The correlation between product price and the number of times a product has been purchased is ", a[0][1])

### Query 10 :

In [ ]:
query10 = """ select *, dense_rank() over(order by total_value desc) as rank_no from
(SELECT 
    order_items.seller_id,
    COUNT(order_items.order_id) AS total_number_of_sales,
    ROUND(SUM(payments.payment_value), 2) AS total_value
FROM
    order_items
        JOIN
    payments ON order_items.order_id = payments.order_id
GROUP BY order_items.seller_id) as a """

cur.execute(query10)

data10 = cur.fetchall()

df10 = pd.DataFrame(data10, columns = ["Seller_id", "Total_Number_of_Orders", "Total_revenue","Rank"])
df10.head()

### Query 11 :

In [ ]:
query11 = """ select *, avg(bill) over(partition by customer_id order by OrderTime rows between 2 preceding and current row ) as mov_avg
from
(select orders.customer_id, orders.order_purchase_timestamp as OrderTime,
payments.payment_value as bill
from payments join orders
on payments.order_id = orders.order_id)  as a """

cur.execute(query11)

data11 = cur.fetchall()

df11 = pd.DataFrame(data11, columns = ["customer_id", "OrderTime", "bill","mov_avg"])
df11.head()

### Query 12 :

In [ ]:
query12 = """ select years, Months, total_revenue,round(sum(total_revenue) over(order by years, Months),2) as cumilative_value from 
(select 
year(orders.order_purchase_timestamp) as years,
month(orders.order_purchase_timestamp) as Months,
-- monthname(orders.order_purchase_timestamp) as Months,
round(sum(payments.payment_value),2) as total_revenue
from orders join payments
on orders.order_id = payments.order_id
group by years,Months
order by years, Months asc) as a; """

cur.execute(query12)

data12 = cur.fetchall()

df12 = pd.DataFrame(data12, columns = ["years", "Months", "Total_revenue","cumulative_value"])
df12

### Query 13 :

In [ ]:
query13 = """ with a as(select 
year(orders.order_purchase_timestamp) as years,
round(sum(payments.payment_value),2) as total_revenue
from orders join payments
on orders.order_id = payments.order_id
group by years
order by years asc)

select *,round((((total_revenue - lag(total_revenue,1) over (order by years)) / lag(total_revenue,1) over (order by years) ) * 100 ),2) as yoy_perc_growth from a; """

cur.execute(query13)

data13 = cur.fetchall()

df13 = pd.DataFrame(data13, columns = ["years", "Total_revenue","YoY % Growth"])
df13

### Query 14 :

In [ ]:
query14 = """with a as (select customers.customer_id,
min(orders.order_purchase_timestamp) first_order
from customers join orders
on customers.customer_id = orders.customer_id
group by customers.customer_id),

b as (select a.customer_id, count(distinct orders.order_purchase_timestamp) next_order
from a join orders
on orders.customer_id = a.customer_id
and orders.order_purchase_timestamp > first_order
and orders.order_purchase_timestamp < 
date_add(first_order, interval 6 month)
group by a.customer_id) 

select 100 * (count( distinct a.customer_id)/ count(distinct b.customer_id)) as retention_rate
from a left join b 
on a.customer_id = b.customer_id ;"""

cur.execute(query14)
data14 = cur.fetchall()

df14 = pd.DataFrame(data14, columns = ["retention_rate"] )
df14

# Query 15 :

In [ ]:
query15 = """ select * from
(select orders.customer_id, 
year(orders.order_purchase_timestamp) as purchase_year,
round(sum(payments.payment_value),2) as total_bill,
dense_rank() over(partition by year(orders.order_purchase_timestamp) order by sum(payments.payment_value) desc ) as customer_ranking
from orders join payments
on orders.order_id = payments.order_id
-- where rankNo <= 3
group by purchase_year,orders.customer_id) as a
where customer_ranking <= 3; """

cur.execute(query15)
data15 = cur.fetchall()

df15 = pd.DataFrame(data15, columns = ["customer_id", "purchase_year","total_bill", "customer_ranking"] )
df15